## loading libraries 

In [1]:
import geemap
import ee
import os

## Authentication 

In [2]:
%%bash
earthengine authenticate --quiet

Einmal anmelden. Alle Google-Produkte nutzen

Mit Ihrem Google-Konto anmelden

[avatar_2x]
E-Mail-Adresse eingeben [                    ] [                    ]
[Weiter] Mein Konto finden
Mit einem anderen Konto anmelden Konto erstellen

Ein Google-Konto für alle Google-Dienste

  • Über Google
  • Datenschutz
  • Nutzungsbedingungen
  • Hilfe

Sprache ändern [‪Deutsch‬                 ]
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=PNukVAn_4HIclQoMLkcuAiaH09w1xlK_qU-_T2xfrB0&code_challenge_method=S256

On the web page, please authorize access to your Earth Engine ac

In [3]:
%%bash
earthengine authenticate --code-verifier=L-Gm53jBAjjMIvnvkcUsPLKBFVnXzwJlpkF57rPQJAE --authorization-code=4/3AHJhhDLsdZmtAcpi31nfGNfB-j-HndzV26FNKM_VPLZ4BnrFVeDreE

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API


Successfully saved authorization token.


## Loading the MAP

In [4]:
Map = geemap.Map(center=[53.868278, 13.268556], zoom=10)
Map

Map(center=[53.868278, 13.268556], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleB…

## Finding the Sentinel-2 image with lowest cloud 

In [22]:
# Add Earth Engine dataset
dataset_S2= ee.ImageCollection('COPERNICUS/S2_SR')


point = ee.Geometry.Point(13.268556, 53.868278)
start = ee.Date('2020-06-01')
finish = ee.Date('2020-06-15')

filteredCollection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(point) \
    .filterDate(start, finish) \
    .sort('CLOUD_COVER', True)

first = filteredCollection.first()
# Define visualization parameters in an object literal.
#vizParams = {'bands': ['B4', 'B3', 'B2'], 'min': 5000, 'max': 15000, 'gamma': 1.3}
vizParams = {"bands": ['B8', 'B4', 'B3'], "max": 4000}
Map.addLayer(first, vizParams, 'Sentinel-2')



## Showing the image information 

In [19]:
# get the image information 

ID = first.id()
print("fileName :", ID.getInfo())
bandNames = first.bandNames()
print("band names: ", bandNames.getInfo())
properties = first.propertyNames()
print("Metadata properties:", properties.getInfo())
date = ee.Date(first.get("system:time_start"))
print("Time Stamp :", date.getInfo())
#print(first.getInfo())

fileName : 20200602T100559_20200602T101204_T32UQE
band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']
Metadata properties: ['DATATAKE_IDENTIFIER', 'AOT_RETRIEVAL_ACCURACY', 'SPACECRAFT_NAME', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'CLOUD_SHADOW_PERCENTAGE', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint', 'VEGETATION_PERCENTAGE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SENSOR_QUALITY', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'FORMAT_CORRECTNESS', 'CLOUD_COVERAGE_ASSESSMENT', 'THIN_CIRRUS_PERCENTAGE', 'system:time_end', 'WATER_VAPOUR_RETRIEVAL_ACCURACY', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'NODATA_PIXEL_PERCENTAGE', 'SENSING_ORBIT_DIRECTION', 'GENERAL_QUALITY', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRE

## Creating the geometry point 

In [16]:
points = ee.Geometry.Point([13.268556, 53.868278])  # create an ee.Geometry 

Map.addLayer(points, {}, 'feature')


## Extracting Pixel values  

* Tip: Pixel values for sentinel-2 should be divided to 10000 
* for future reference, refere to https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands and check the scale  


In [17]:
# use these bands for extraction 

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

# load coordinate of  points
lable = 'STEROPES'
# Overlay the points on the imagery to get training.
pValues = first.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [lable],
    'scale': 10
})

Map.centerObject(points, 10)
Map.addLayer(points, {'color': "yellow"}, 'Extracted points')

pV = pValues.first()
print(pV.getInfo())   


{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 374, 'B11': 1553, 'B12': 829, 'B2': 414, 'B3': 694, 'B4': 445, 'B5': 1203, 'B6': 3669, 'B7': 4243, 'B8': 4452, 'B8A': 4706, 'B9': 5206}}
